In [76]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict
import os

load_dotenv()

True

In [77]:
LLM = ChatOpenAI(
    model="xiaomi/mimo-v2-flash:free",
    api_key=os.environ.get('open_router_api_key'),
    base_url="https://openrouter.ai/api/v1",
)

In [78]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    score: int

In [79]:
def create_outline(state:BlogState):
    title = state['title']
    
    prompt = f'Generate an outline for blog on the topic {title}'
    
    outline = LLM.invoke(prompt).content
    
    state['outline'] = outline
    
    return state

In [80]:
def create_blog(state):
    title = state['title']
    outline = state['outline']
    
    prompt = f"Write a detailed proffesional blog on title {title}, outline {outline}"
    
    blog = LLM.invoke(prompt).content
    
    state['content'] = blog
    
    return state

In [81]:
def blog_eval(state):
    title = state['title']
    outline = state['outline']
    blog = state['content']
    
    prompt = f"""
         act as a  professional blog evaluator, Score my blog out of 10,
            Blog data title{title} outline {outline} content {blog}
    """
    score = LLM.invoke(prompt)
    state['score'] = score
    
    return state
    

In [82]:
graph = StateGraph(BlogState)

graph.add_node('create_outline', create_outline)
graph.add_node('create_blog',create_blog)
graph.add_node('blog_eval', blog_eval)


graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog','blog_eval')
graph.add_edge('blog_eval',END)

workflow = graph.compile()

In [83]:
initial_state = {'title':'Need to Create an Indian opensource LLM models'}
final_state = workflow.invoke(initial_state)


In [86]:
print(final_state)

{'title': 'Need to Create an Indian opensource LLM models', 'outline': 'Of course! Here is a comprehensive and structured outline for a blog post on the topic "The Need to Create Indian Open-Source LLM Models."\n\nThis outline is designed to be persuasive, informative, and actionable, guiding the reader from the problem to the solution and the call to action.\n\n---\n\n### **Blog Title Options:**\n\n*   **Primary:** Beyond Silicon Valley: The Critical Need for Indian Open-Source LLMs\n*   **Secondary:** Building Our Own AI: Why India Must Champion an Open-Source LLM\n*   **Catchy:** The Desi AI Revolution: It\'s Time to Code Our Own Large Language Model\n\n---\n\n### **Blog Outline**\n\n**Introduction: The AI We Use Isn\'t Built for Us**\n\n*   **Hook:** Start with a relatable scenario. Ask the reader if they\'ve ever been frustrated by an AI (like a voice assistant or chatbot) that doesn\'t understand Hinglish, regional slang, or the cultural context of a simple Indian reference.\n*  

In [88]:
print(final_state['score'])

content='Of course. As a professional blog evaluator, I will provide a comprehensive score and a detailed critique of your blog post based on key industry-standard metrics.\n\n---\n\n### **Overall Score: 9.25 / 10**\n\nThis is an exceptional piece of writing. It is well-structured, persuasive, and addresses a highly relevant and timely topic with clarity and strategic depth. It reads like a thought leadership piece you would find in a major publication like *The Ken*, *YourStory*, or a dedicated tech policy journal. The score reflects its near-perfect execution, with only minor areas for enhancement.\n\n---\n\n### **Detailed Evaluation Breakdown**\n\nHere is a scorecard that breaks down the evaluation into key components:\n\n| Metric | Score (out of 10) | Commentary |\n| :--- | :--- | :--- |\n| **Clarity & Thesis** | 9.5 | The argument is exceptionally clear from the first paragraph. The thesis is strong, well-defined, and consistently reinforced. |\n| **Structure & Flow** | 10 | The o